In [1]:
import numpy as np
import pickle
import keras.backend as K

from keras.layers import  GlobalAveragePooling1D, Input, Activation, MaxPooling1D, BatchNormalization, Dense, Dropout, Conv1D,GlobalMaxPooling1D
from keras.layers import GRU,AveragePooling1D,CuDNNGRU
from keras.layers.merge import Concatenate
from keras.models import Model 
from keras.callbacks import EarlyStopping,ModelCheckpoint
from gensim.models import Word2Vec 

import keras.backend.tensorflow_backend as KTF
import tensorflow as tf
import os


os.environ["CUDA_VISIBLE_DEVICES"] = "0"

config = tf.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)

KTF.set_session(sess)

Using TensorFlow backend.
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/xhh/anaconda3/envs/py36/lib/p

In [2]:
class TermVectors:
    def __init__(self):
        self.termVectorDict=dict()
        self.terms=None
        self.vectors=None

    def parse_term_embedding_file(self,file_path):
        with open(file_path) as f:
            lines=f.readlines()
            self.vectors=lines[1:] # 第一个是term的数量，不是termID
            terms = []
            for line in lines[1:]:
                term=line.split()[0]
                terms.append(term)
            self.terms = terms

    def str_to_vector(self,Str):
        Str = Str.strip('\n')
        nums = Str.split()
        vec = []
        for num in nums:
            vec.append(float(num))
        return np.array(vec)

    def construct_term_vector_dict(self):
        for term in self.terms:
            termindex = self.terms.index(term)
            line=self.vectors[termindex]
            s =line[len(term):].lstrip(' ')
            s_vec =self.str_to_vector(s)
            self.termVectorDict[term]=s_vec

In [3]:
# construct Node2Vec termVector
BP_TERM_EMB_FILE_PATH = '../Node2Vec/EDGELIST/BP.emb'
CC_TERM_EMB_FILE_PATH = '../Node2Vec/EDGELIST/CC.emb'
MF_TERM_EMB_FILE_PATH = '../Node2Vec/EDGELIST/MF.emb'
Node2Vec_dim = 300


BPTermVectors = TermVectors()
BPTermVectors.parse_term_embedding_file(BP_TERM_EMB_FILE_PATH)
BPTermVectors.construct_term_vector_dict()
print(len(BPTermVectors.termVectorDict))

CCTermVectors = TermVectors()
CCTermVectors.parse_term_embedding_file(CC_TERM_EMB_FILE_PATH)
CCTermVectors.construct_term_vector_dict()
print(len(CCTermVectors.termVectorDict))

MFTermVectors = TermVectors()
MFTermVectors.parse_term_embedding_file(MF_TERM_EMB_FILE_PATH)
MFTermVectors.construct_term_vector_dict()
print(len(MFTermVectors.termVectorDict))

29699
4202
11148


In [4]:
# read go.obo obtain ontology type
id_type_dicts = {}
obo_file = '../cross-species/go.obo'
fp=open(obo_file,'r')
obo_txt=fp.read()
fp.close()
obo_txt=obo_txt[obo_txt.find("[Term]")-1:]
obo_txt=obo_txt[:obo_txt.find("[Typedef]")]
# obo_dict=parse_obo_txt(obo_txt)
id_type_dicts = {}
for Term_txt in obo_txt.split("[Term]\n"):
    if not Term_txt.strip():
        continue
    name = ''
    ids = []
    for line in Term_txt.splitlines():
        if   line.startswith("id: "):
            ids.append(line[len("id: "):])     
        elif line.startswith("namespace: "):
             name=line[len("namespace: "):]
        elif line.startswith("alt_id: "):
            ids.append(line[len("alt_id: "):])
    
    for t_id in ids:
        id_type_dicts[t_id] = name

In [5]:
import numpy as np
alphabet = np.array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
                     'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'])

def label_sequence(line, MAX_SEQ_LEN, smi_ch_ind):
	X = np.zeros(MAX_SEQ_LEN)

	for i, ch in enumerate(line[:MAX_SEQ_LEN]):
		X[i] = smi_ch_ind[ch]

	return X #.tolist()

def letter_one_hot(aa):
    one_hot = np.zeros(20)
    for idx, letter in enumerate(alphabet):
        if aa == letter:
            one_hot[idx] = 1
            return one_hot


# Convert an entire protein to one-hot representation.
def protein_one_hot(protein_sequence, MAX_SEQ_LEN):
    #  Remove non-specific AA codes (very few are actually present in this dataset)
    protein_sequence = protein_sequence.replace('B', '')
    protein_sequence = protein_sequence.replace('J', '')
    protein_sequence = protein_sequence.replace('O', '')
    protein_sequence = protein_sequence.replace('U', '')
    protein_sequence = protein_sequence.replace('X', '')
    protein_sequence = protein_sequence.replace('Z', '')
    one_hot_seq = np.zeros( (MAX_SEQ_LEN, 20))
    for idx, aa in enumerate(protein_sequence[:MAX_SEQ_LEN]):
        one_hot_seq[idx, :] = letter_one_hot(aa)
    return one_hot_seq


In [6]:
import keras
feature_len = 768
max_go_len = 512
max_seq_len = 2000
max_node_len = 256
from six.moves import cPickle as pickle #for performance

 
def save_dict(di_, filename_):
    with open(filename_, 'wb') as f:
        pickle.dump(di_, f)

def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pickle.load(f)
    return ret_di



class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self,  ppi_pair_file, batch_size=128):
        'Initialization' 
        self.batch_size = batch_size
        self.ppi_pair_file = ppi_pair_file
         
        self.max_seqlen = max_seq_len
        self.max_golen = max_go_len
        self.max_node_len = max_node_len
        self.protein2go =  load_dict('yeast_data/protein2go_dicts.pkl')
 
        self.protein2seq = load_dict('yeast_data/protein2seq_dicts.pkl')
        self.read_ppi()
         
        self.protein2onehot = {}
        self.onehot_seqs()
        self.prot2nodevec = {}
        self.prot2nodevec_fun()
        self.on_epoch_end()
    
    def prot2nodevec_fun(self):
        for key, value in self.protein2go.items():
            X_go1 =  np.zeros((1,Node2Vec_dim))
            allgos = value.split(';') 
            allgos = list(set(allgos))
            count = 0
            for  go in  allgos:
                if go.startswith('GO'):
                    if id_type_dicts[go] == '':
                        termVectors = BPTermVectors
                        term_ID=go[3:].lstrip('0')
                        feature = termVectors.termVectorDict[term_ID]
                    elif id_type_dicts[go] == '':
                        termVectors = CCTermVectors
                        term_ID=go[3:].lstrip('0')
                        feature = termVectors.termVectorDict[term_ID]
                    elif id_type_dicts[go] == '':
                        termVectors = MFTermVectors
                        term_ID=go[3:].lstrip('0')
                        feature = termVectors.termVectorDict[term_ID]
                    else:
                        feature = np.zeros((1,Node2Vec_dim))
                    
                     
                    if count + feature.shape[0] > max_go_len:
                        break
                    X_go1 = np.concatenate((X_go1,feature ))    
                    count += feature.shape[0]
            self.prot2nodevec[key] =  X_go1[1:]   
        
    
    def read_ppi(self):
        with open(self.ppi_pair_file, 'r') as f:
            self.ppi_pairs  =  f.readlines()
            
    
    def onehot_seqs(self):
        for key, value in self.protein2seq.items():
            self.protein2onehot[key] =  protein_one_hot(value, self.max_seqlen) 

    

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.ppi_pairs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    
     
            
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.ppi_pairs))
         
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization

        
        y = np.empty((self.batch_size))
        X_seq1 = np.empty((self.batch_size, self.max_seqlen,20))
        X_seq2 = np.empty((self.batch_size, self.max_seqlen,20))
        
        
        X_node1 = np.empty((self.batch_size, self.max_node_len,Node2Vec_dim))
        X_node2 = np.empty((self.batch_size, self.max_node_len,Node2Vec_dim))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            ppi_pair = self.ppi_pairs[ID]
            p1, p2, label = ppi_pair.rstrip().split('\t')
            if label == '1':
                y[i] = 1
            else:
                y[i] = 0
                
             
            
            X_seq1[i] =  self.protein2onehot[p1]
            X_seq2[i] =  self.protein2onehot[p2]
            
            prot1emb_node = self.prot2nodevec[p1]
            X_node1[i,:prot1emb_node.shape[0]] = prot1emb_node
            
            prot2emb_node = self.prot2nodevec[p2]
            X_node2[i,:prot2emb_node.shape[0]] = prot2emb_node
            
            
            
     
        return [  X_node1, X_node2, X_seq1, X_seq2] ,  y



    def all_data(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization

         
        y = np.empty((len(list_IDs_temp)))
        
        X_seq1 = np.empty((len(list_IDs_temp), self.max_seqlen,20))
        X_seq2 = np.empty((len(list_IDs_temp), self.max_seqlen,20))
        
        
        X_node1 = np.empty((len(list_IDs_temp), self.max_node_len,Node2Vec_dim))
        X_node2 = np.empty((len(list_IDs_temp), self.max_node_len,Node2Vec_dim))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            ppi_pair = self.ppi_pairs[ID]
            p1, p2, label = ppi_pair.rstrip().split('\t')
            if label == '1':
                y[i] = 1
            else:
                y[i] = 0
            
            
            
            X_seq1[i] =  self.protein2onehot[p1]
            X_seq2[i] =  self.protein2onehot[p2]
            
            prot1emb_node = self.prot2nodevec[p1]
            X_node1[i,:prot1emb_node.shape[0]] = prot1emb_node
            
            prot2emb_node = self.prot2nodevec[p2]
            X_node2[i,:prot2emb_node.shape[0]] = prot2emb_node
            
  
        return [  X_node1, X_node2, X_seq1, X_seq2] ,  y

In [7]:
from keras import backend as K, initializers, regularizers, constraints
from keras.engine.topology import Layer


def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        # todo: check that this is correct
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)


class Attention(Layer):
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True,
                 return_attention=False,
                 **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Note: The layer has been tested with Keras 1.x
        Example:
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2 - Get the attention scores
            hidden = LSTM(64, return_sequences=True)(words)
            sentence, word_scores = Attention(return_attention=True)(hidden)
        """
        self.supports_masking = True
        self.return_attention = return_attention
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        eij = dot_product(x, self.W)

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        weighted_input = x * K.expand_dims(a)

        result = K.sum(weighted_input, axis=1)

        if self.return_attention:
            return [result, a]
        return result

    def compute_output_shape(self, input_shape):
        if self.return_attention:
            return [(input_shape[0], input_shape[-1]),
                    (input_shape[0], input_shape[1])]
        else:
            return input_shape[0], input_shape[-1]

In [8]:
from keras.layers import   Embedding
from keras.layers import  GRU, Bidirectional, CuDNNGRU, Lambda, dot, Flatten, CuDNNLSTM, Add
from keras.layers.merge import concatenate

def inception_block(input_tensor, output_size):
    """"""
    con1d_filters = int(output_size/4)
    y = Conv1D(con1d_filters, 3, activation="relu", padding='same')(input_tensor)
    x1 = Conv1D(con1d_filters, 5, activation="relu", padding='same')(y)

    y = Conv1D(con1d_filters, 1, activation="relu", padding='valid')(input_tensor)
    x2 = Conv1D(con1d_filters, 3, activation="relu", padding='same')(y)

    x3 = Conv1D(con1d_filters, 3, activation="relu", padding='same')(input_tensor)
    x4 = Conv1D(con1d_filters, 1, activation="relu", padding='same')(input_tensor)

    y = Concatenate()([x1, x2, x3, x4])
#     y = MaxPooling1D(2)(mix0)
    # y = AveragePooling1D()(mix0)
#     y = BatchNormalization()(y)

    return y


 
def build_cnn_gru_model(input_x, con_filters, gru_units):
    x = inception_block(input_x,con_filters )
    x = Dropout(0.3)(x)
    x_gru = Bidirectional(CuDNNGRU(gru_units, return_sequences=True))(input_x)
    x_gru = Dropout(0.3)(x_gru)
     
    x_a = GlobalAveragePooling1D()(x)
    x_b = GlobalMaxPooling1D()(x)
    x_c = Attention()(x)
    x_gru_a = GlobalAveragePooling1D()(x_gru)
    x_gru_b = GlobalMaxPooling1D()(x_gru)
    x_gru_c = Attention()(x_gru)
    x = Concatenate()([ x_a, x_b, x_c, x_gru_c, x_gru_b,  x_gru_a])
    x = Dense(256)(x)
    return x


def build_cnn_model(input_x, con_filters, gru_units):
    x = inception_block(input_x,con_filters )
    x = Dropout(0.3)(x)
     
    x_a = GlobalAveragePooling1D()(x)
    x_b = GlobalMaxPooling1D()(x)
    x_c = Attention()(x)
    
    x = Concatenate()([ x_a, x_b, x_c])
    x = Dense(256)(x)
    return x 


def build_model():
    con_filters = 128
    gru_units = 64
    
    
    left_input_node = Input(shape=(max_node_len,Node2Vec_dim))
    right_input_node = Input(shape=(max_node_len,Node2Vec_dim))
    
    
    left_input_seq = Input(shape=(max_seq_len,20))
    right_input_seq = Input(shape=(max_seq_len,20))
    
     

     
    
    left_x_seq = build_cnn_gru_model(left_input_seq, con_filters, gru_units)
    right_x_seq = build_cnn_gru_model(right_input_seq, con_filters, gru_units)
    
    left_x_node = build_cnn_model(left_input_node, con_filters//2, gru_units)
    right_x_node = build_cnn_model(right_input_node, con_filters//2,gru_units)
    
      
     
    
    
   
   
    x =   Concatenate()([ left_x_node,  right_x_node, left_x_seq, right_x_seq])
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(512, activation='relu')(x)
  
    x = Dense(1)(x)
    output = Activation('sigmoid')(x)
    # model = Model([left_input_go, right_input_go], output)
  
    model = Model([  left_input_node, right_input_node,     left_input_seq, right_input_seq], output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


 

model = build_model()
model.summary()
# siamese_a = create_share_model()
# siamese_a.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 300)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 256, 300)     0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 2000, 20)     0                                            
___________________________________________________________________________

In [9]:
from sklearn.model_selection import StratifiedKFold
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
dataset_name = 'yeast'
for rep in range(1):
    n_splits = 5
    TPRs =  np.zeros(n_splits)
    FPRs = np.zeros(n_splits)
    Precs = np.zeros(n_splits)
    ACCs = np.zeros(n_splits)
    F1s = np.zeros(n_splits)
    MCCs = np.zeros(n_splits)
    AUCs = np.zeros(n_splits)
    skf = StratifiedKFold(n_splits=n_splits)
    count = 0
    for split in range(0,5):
        train_pairs_file = 'yeast_data/train'+str(rep)+'-'+str(split)
        test_pairs_file = 'yeast_data/test'+str(rep)+'-'+str(split)
        valid_pairs_file = 'yeast_data/valid'+str(rep)+'-'+str(split)

        batch_size = 128
        train_generator = DataGenerator(   train_pairs_file,batch_size = batch_size )
        test_generator = DataGenerator(   test_pairs_file,batch_size = batch_size)
        valid_generator = DataGenerator(   valid_pairs_file,batch_size = batch_size)
         
        # model = build_model_without_att()
        model = build_model()
        save_model_name = 'yeast_data/node_seq'+str(rep)+'-'+str(split) + '.hdf5'
        
        earlyStopping = EarlyStopping(monitor='val_acc', patience=15, verbose=0, mode='max')
        save_checkpoint = ModelCheckpoint(save_model_name, save_best_only=True, monitor='val_acc', mode='max', save_weights_only=True)

         
        # validation_data = (valid_X, valid_Y),  verbose=1,callbacks=[earlyStopping, save_checkpoint]
        hist = model.fit_generator(generator=train_generator,
                   validation_data=valid_generator, 
                   epochs = 100,verbose=1,callbacks=[earlyStopping, save_checkpoint] )
         
        
        # model = load_model(save_model_name)
        model.load_weights(save_model_name)
        with open(test_pairs_file, 'r') as f:
            test_ppi_pairs  =  f.readlines()

        test_len = len(test_ppi_pairs) 
        list_IDs_temp = np.arange(test_len)

        test_x, y_test = test_generator.all_data(list_IDs_temp)

        y_pred_prob = model.predict(test_x)

       
        y_pred = (y_pred_prob > 0.5)
        auc = metrics.roc_auc_score(y_test, y_pred_prob) 
        f1 = f1_score(y_test, y_pred)
        pre = precision_score(y_test, y_pred)
        acc = accuracy_score(y_test, y_pred)

        precision, recall, _thresholds = metrics.precision_recall_curve(y_test, y_pred_prob)
        pr_auc = metrics.auc(recall, precision)
        mcc = matthews_corrcoef(y_test, y_pred)

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        total=tn+fp+fn+tp
        sen = float(tp)/float(tp+fn)
        sps = float(tn)/float((tn+fp))

        tpr = float(tp)/float(tp+fn)
        fpr = float(fp)/float((tn+fp))
        print('--------------------------\n')
        print ('AUC: %f' % auc)
        print ('ACC: %f' % acc) 
        # print("PRAUC: %f" % pr_auc)
        print ('MCC : %f' % mcc)
        # print ('SEN: %f' % sen)
        # print ('SEP: %f' % sps)
        print('TPR:%f'%tpr)
        print('FPR:%f'%fpr)
        print('Pre:%f'%pre)
        print('F1:%f'%f1)
        print('--------------------------\n')
        TPRs[count] = tpr
        FPRs[count] = fpr
        Precs[count] =pre
        ACCs[count] =acc
        F1s[count] =f1
        MCCs[count] =mcc
        AUCs[count] =auc
        count += 1
        del test_x
        del y_test
    print ('mean AUC: %f' % np.mean(AUCs))
    print ('mean ACC: %f' % np.mean(ACCs)) 
    print ('mean MCC : %f' % np.mean(MCCs))
    print('mean TPR:%f'% np.mean(TPRs))
    print('mean FPR:%f'% np.mean(FPRs))
    print('mean Pre:%f'% np.mean(Precs))
    print('mean F1:%f'% np.mean(F1s))
#     np.savez('yeast_go_seq'+str(rep), AUCs=AUCs, ACCs=ACCs, MCCs=MCCs, TPRs = TPRs, FPRs=FPRs, Precs=Precs, F1s=F1s)







Epoch 1/100
55/55 [==============================] - 44s 796ms/step - loss: 0.6727 - acc: 0.6246 - val_loss: 0.6649 - val_acc: 0.6172
Epoch 2/100
55/55 [==============================] - 31s 563ms/step - loss: 0.6634 - acc: 0.6260 - val_loss: 0.6638 - val_acc: 0.6172
Epoch 3/100
55/55 [==============================] - 33s 600ms/step - loss: 0.6623 - acc: 0.6260 - val_loss: 0.6622 - val_acc: 0.6172
Epoch 4/100
55/55 [==============================] - 33s 596ms/step - loss: 0.6589 - acc: 0.6260 - val_loss: 0.6620 - val_acc: 0.6172
Epoch 5/100
55/55 [==============================] - 33s 592ms/step - loss: 0.6459 - acc: 0.6368 - val_loss: 0.6396 - val_acc: 0.6695
Epoch 6/100
55/55 [==============================] - 32s 590ms/step - loss: 0.6581 - acc: 0.6304 - val_loss: 0.6661 - val_acc: 0.6172
Epoch 7/100
55/55 [==============================] - 32s 576ms/step - loss: 0.6557 - acc: 0.6253 - val_loss: 0.6568 - val_acc: 0.6196
Epoch 8/100
55/55 [==============================] - 33s 603ms

Epoch 62/100
55/55 [==============================] - 33s 592ms/step - loss: 0.0939 - acc: 0.9632 - val_loss: 0.4590 - val_acc: 0.8798
Epoch 63/100
55/55 [==============================] - 31s 570ms/step - loss: 0.1026 - acc: 0.9619 - val_loss: 0.5766 - val_acc: 0.8654
Epoch 64/100
55/55 [==============================] - 31s 569ms/step - loss: 0.0847 - acc: 0.9682 - val_loss: 0.5275 - val_acc: 0.8804
Epoch 65/100
55/55 [==============================] - 33s 595ms/step - loss: 0.1082 - acc: 0.9605 - val_loss: 0.4861 - val_acc: 0.8780
Epoch 66/100
55/55 [==============================] - 31s 565ms/step - loss: 0.0955 - acc: 0.9670 - val_loss: 0.5994 - val_acc: 0.8768
Epoch 67/100
55/55 [==============================] - 32s 590ms/step - loss: 0.0842 - acc: 0.9690 - val_loss: 0.5445 - val_acc: 0.8708
Epoch 68/100
55/55 [==============================] - 31s 562ms/step - loss: 0.0955 - acc: 0.9641 - val_loss: 0.5471 - val_acc: 0.8413
Epoch 69/100
55/55 [==============================] - 3

55/55 [==============================] - 30s 545ms/step - loss: 0.1632 - acc: 0.9324 - val_loss: 0.4012 - val_acc: 0.8552
Epoch 43/100
55/55 [==============================] - 30s 547ms/step - loss: 0.1361 - acc: 0.9419 - val_loss: 0.4626 - val_acc: 0.8558
Epoch 44/100
55/55 [==============================] - 30s 550ms/step - loss: 0.1488 - acc: 0.9375 - val_loss: 0.4126 - val_acc: 0.8654
Epoch 45/100
55/55 [==============================] - 29s 536ms/step - loss: 0.1400 - acc: 0.9425 - val_loss: 0.3921 - val_acc: 0.8642
Epoch 46/100
55/55 [==============================] - 29s 529ms/step - loss: 0.1360 - acc: 0.9442 - val_loss: 0.4145 - val_acc: 0.8582
Epoch 47/100
55/55 [==============================] - 28s 507ms/step - loss: 0.1480 - acc: 0.9398 - val_loss: 0.4336 - val_acc: 0.8624
Epoch 48/100
55/55 [==============================] - 28s 503ms/step - loss: 0.1261 - acc: 0.9477 - val_loss: 0.4653 - val_acc: 0.8624
Epoch 49/100
55/55 [==============================] - 28s 515ms/step

55/55 [==============================] - 39s 715ms/step - loss: 0.6772 - acc: 0.6190 - val_loss: 0.6734 - val_acc: 0.6376
Epoch 2/100
55/55 [==============================] - 30s 552ms/step - loss: 0.6645 - acc: 0.6227 - val_loss: 0.6596 - val_acc: 0.6376
Epoch 3/100
55/55 [==============================] - 31s 557ms/step - loss: 0.6644 - acc: 0.6227 - val_loss: 0.6599 - val_acc: 0.6376
Epoch 4/100
55/55 [==============================] - 31s 569ms/step - loss: 0.6593 - acc: 0.6227 - val_loss: 0.6572 - val_acc: 0.6376
Epoch 5/100
55/55 [==============================] - 31s 564ms/step - loss: 0.6837 - acc: 0.6230 - val_loss: 0.6846 - val_acc: 0.6376
Epoch 6/100
55/55 [==============================] - 33s 594ms/step - loss: 0.6681 - acc: 0.6156 - val_loss: 0.6537 - val_acc: 0.6376
Epoch 7/100
55/55 [==============================] - 31s 557ms/step - loss: 0.6639 - acc: 0.6227 - val_loss: 0.6583 - val_acc: 0.6376
Epoch 8/100
55/55 [==============================] - 31s 571ms/step - loss

Epoch 62/100
55/55 [==============================] - 30s 553ms/step - loss: 0.1267 - acc: 0.9536 - val_loss: 0.3942 - val_acc: 0.8732
Epoch 63/100
55/55 [==============================] - 32s 578ms/step - loss: 0.1014 - acc: 0.9598 - val_loss: 0.4493 - val_acc: 0.8690
Epoch 64/100
55/55 [==============================] - 32s 586ms/step - loss: 0.0963 - acc: 0.9628 - val_loss: 0.4389 - val_acc: 0.8726
Epoch 65/100
55/55 [==============================] - 33s 594ms/step - loss: 0.0947 - acc: 0.9631 - val_loss: 0.4745 - val_acc: 0.8612
Epoch 66/100
55/55 [==============================] - 32s 586ms/step - loss: 0.0881 - acc: 0.9658 - val_loss: 0.4369 - val_acc: 0.8660
Epoch 67/100
55/55 [==============================] - 32s 581ms/step - loss: 0.0879 - acc: 0.9692 - val_loss: 0.4636 - val_acc: 0.8762
Epoch 68/100
55/55 [==============================] - 33s 599ms/step - loss: 0.0865 - acc: 0.9675 - val_loss: 0.4936 - val_acc: 0.8510
Epoch 69/100
55/55 [==============================] - 3

55/55 [==============================] - 30s 551ms/step - loss: 0.2831 - acc: 0.8808 - val_loss: 0.4175 - val_acc: 0.8119
Epoch 22/100
55/55 [==============================] - 31s 566ms/step - loss: 0.2727 - acc: 0.8810 - val_loss: 0.5258 - val_acc: 0.7831
Epoch 23/100
55/55 [==============================] - 31s 570ms/step - loss: 0.2665 - acc: 0.8842 - val_loss: 0.4260 - val_acc: 0.8113
Epoch 24/100
55/55 [==============================] - 33s 593ms/step - loss: 0.2667 - acc: 0.8918 - val_loss: 0.4597 - val_acc: 0.8113
Epoch 25/100
55/55 [==============================] - 32s 583ms/step - loss: 0.2410 - acc: 0.8994 - val_loss: 0.4359 - val_acc: 0.8185
Epoch 26/100
55/55 [==============================] - 31s 571ms/step - loss: 0.2455 - acc: 0.8960 - val_loss: 0.4577 - val_acc: 0.8101
Epoch 27/100
55/55 [==============================] - 32s 580ms/step - loss: 0.2339 - acc: 0.9013 - val_loss: 0.4165 - val_acc: 0.8077
Epoch 28/100
55/55 [==============================] - 31s 573ms/step

Epoch 1/100
55/55 [==============================] - 42s 770ms/step - loss: 0.6747 - acc: 0.6159 - val_loss: 0.6555 - val_acc: 0.6358
Epoch 2/100
55/55 [==============================] - 31s 560ms/step - loss: 0.6647 - acc: 0.6229 - val_loss: 0.6548 - val_acc: 0.6358
Epoch 3/100
55/55 [==============================] - 31s 556ms/step - loss: 0.6654 - acc: 0.6229 - val_loss: 0.6597 - val_acc: 0.6358
Epoch 4/100
55/55 [==============================] - 31s 560ms/step - loss: 0.6595 - acc: 0.6229 - val_loss: 0.6559 - val_acc: 0.6358
Epoch 5/100
55/55 [==============================] - 32s 586ms/step - loss: 0.6594 - acc: 0.6203 - val_loss: 0.6707 - val_acc: 0.6605
Epoch 6/100
55/55 [==============================] - 31s 568ms/step - loss: 0.6298 - acc: 0.6626 - val_loss: 0.9488 - val_acc: 0.6358
Epoch 7/100
55/55 [==============================] - 30s 544ms/step - loss: 0.6615 - acc: 0.6342 - val_loss: 0.6446 - val_acc: 0.6358
Epoch 8/100
55/55 [==============================] - 29s 533ms

Epoch 62/100
55/55 [==============================] - 33s 592ms/step - loss: 0.0889 - acc: 0.9645 - val_loss: 0.5291 - val_acc: 0.8720
Epoch 63/100
55/55 [==============================] - 31s 558ms/step - loss: 0.1061 - acc: 0.9551 - val_loss: 0.4972 - val_acc: 0.8768
Epoch 64/100
55/55 [==============================] - 31s 567ms/step - loss: 0.1145 - acc: 0.9548 - val_loss: 0.5734 - val_acc: 0.8594
Epoch 65/100
55/55 [==============================] - 32s 573ms/step - loss: 0.1187 - acc: 0.9530 - val_loss: 0.4890 - val_acc: 0.8810
Epoch 66/100
55/55 [==============================] - 32s 580ms/step - loss: 0.0853 - acc: 0.9673 - val_loss: 0.6290 - val_acc: 0.8654
Epoch 67/100
55/55 [==============================] - 32s 586ms/step - loss: 0.0995 - acc: 0.9614 - val_loss: 0.4528 - val_acc: 0.8786
Epoch 68/100
55/55 [==============================] - 32s 579ms/step - loss: 0.1013 - acc: 0.9622 - val_loss: 0.6175 - val_acc: 0.8672
--------------------------

AUC: 0.960750
ACC: 0.923971

In [11]:
np.savez('yeast_seq_node2vec'+str(rep), AUCs=AUCs, ACCs=ACCs, MCCs=MCCs, TPRs = TPRs, FPRs=FPRs, Precs=Precs, F1s=F1s)
